# **Week 3: Challenge Activity Part 1**
In this lab work, we will applying LIME to explain black-box image classifiers.
<br>
<br>
**LIME**<br>
Generally, deep learning models are much more efficient than conventional ML models on image data as these models have the ability to perform auto feature extraction. They can extract complex low-level features such as stripes, edges, contours, corners and motifs, and even higher-level features such as larger shapes and certain parts of the object. These higher-level features are usually referred to as Regions of Interest (RoI) in the image, or superpixels, as they are collections of pixels of the image that cover a particular area of the image. Now, the low-level features are not human-interpretable, but the high-level features are human-interpretable, as any non-technical end user will relate to the images with respect to the higher-level features. LIME also works in a similar fashion. The algorithm tries to highlight the superpixels in images that contribute positively or negatively to the model's decision-making process hence providing explainability. Further information : [LIME GitHub](https://github.com/marcotcr/lime)
<br>
<br>
**Please download shark image file from [Task 3 in Challenge Activity](https://canvas.hull.ac.uk/courses/75587/discussion_topics/481340?module_item_id=1375372) and save your Gdrive.**

In [1]:
# Install the following libraries your local environment, if not already installed.
!pip install lime

In [2]:
# Importing necessary libararies

import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as c_map
from IPython.display import Image, display
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.xception import Xception, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

import lime
from lime import lime_image
from lime import submodular_pick

from skimage.segmentation import mark_boundaries

np.random.seed(123)

## **Loading the data**
Since we are more interested to check how black-box image classifiers can be explained using LIME, we will focus only on the inference part. Let us load any generic image data. For this example, we will take the data from this source: https://i.imgur.com/1Phh6hv.jpeg

In [3]:
# Connect to your Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
image_path = "c:/users/hlmas/Desktop/MSc/Ethics/MSc-Ethics/shark.jpg" # Please change your filepath you saved the data.
IMG_SIZE = (299, 299)

# Display the image.
display(Image(filename=image_path, width=IMG_SIZE[0], height=IMG_SIZE[1]))

In [ ]:
def transform_image(image_path, size):
    '''
    Function to transform an image to normalized numpy array
    '''
    img = image.load_img(image_path, target_size=size)
    img = image.img_to_array(img)# Transforming the image to get the shape as [channel, height, width]
    img = np.expand_dims(img, axis=0) # Adding dimension to convert array into a batch of size (1,299,299,3)
    img = img/255.0 # normalizing the image to keep within the range of 0.0 to 1.0

    return img

normalized_img = transform_image(image_path, IMG_SIZE)

## **Defining the model**
For this example, we are not training a model from scratch, but rather defining a pretrained Tensorflow Xception model as our black-box Deep Learning model which we will be explaining using the LIME framework.

In [ ]:
# Define and initialise a model.
model = Xception(weights="imagenet")

In [ ]:
# Predict class of the image
def get_model_predictions(data):
    model_prediction = model.predict(data)
    print(f"The predicted class is : {decode_predictions(model_prediction, top=1)[0][0][1]}")
    return decode_predictions(model_prediction, top=1)[0][0][1] # get class name

plt.imshow(normalized_img[0])
pred_orig = get_model_predictions(normalized_img)

The image is predicted as Tiger Shark which is the correct prediction and the black-box model is successfully able to give the correct prediction. Now, let us even take a look at the top 5 predictions along with the model confidence.

In [ ]:
model_prediction = model.predict(normalized_img)
top5_pred = decode_predictions(model_prediction, top=5)[0]
for pred in top5_pred:
    print(pred[1])

As we see, although the model is well trained to produce the correct prediction, but there are chances that the model is not just looking into main object in the image but as well as the surrounding background. This is evident from the prediction of scuba_driver present in the top 5 prediction list. So, it is important for us understand, the key components or parts of the image the model is looking into to make the prediction.

# **Model Explanation with LIME**
Now, we will use the LIME framework to identify "super-pixels" or image segments used by the model to predict the outcome.

In [ ]:
explainer = lime_image.LimeImageExplainer()

In [ ]:
exp = explainer.explain_instance(normalized_img[0],
                                 model.predict,
                                 top_labels=5,
                                 hide_color=0, # color code to mask
                                 num_samples=1000) # number of pertutated images

Our explainer object is ready, but let us visualize the various explanation segments created by the LIME algorithm.

In [ ]:
plt.imshow(exp.segments)
plt.axis('off')
plt.show()

Now, let us use the top segments or super pixels to identify the region of interest of the image used by the model to make its prediction.

In [ ]:
def generate_prediction_sample(exp, exp_class, weight = 0.1, show_positive = True, hide_background = True):
    '''
    Method to display and highlight super-pixels used by the black-box model to make predictions
    '''
    image, mask = exp.get_image_and_mask(exp_class,
                                         positive_only=show_positive,
                                         num_features=6, # number of super-pixels
                                         hide_rest=hide_background,
                                         min_weight=weight
                                        )
    plt.imshow(mark_boundaries(image, mask))
    plt.axis('off')
    plt.show()

In [ ]:
generate_prediction_sample(exp, exp.top_labels[0], show_positive = True, hide_background = True)

In [ ]:
generate_prediction_sample(exp, exp.top_labels[0], show_positive = True, hide_background = False)

As we can see from the above image that the model was able to identify the correct region, which does indicate the correct prediction of the outcome by the model.

In [ ]:
generate_prediction_sample(exp, exp.top_labels[0], show_positive = False, hide_background = False)

The above samples show us how we can hide or show the background along with the super-pixels or even outline or highlight the super-pixels to identify the region of interest used by the model to make the prediction. What we see from here does make sense, and does allow us to increase trust towards black-box models. We can also form a heat-map to show how important each super-pixel is to get some more granular explaianbility.

In [ ]:
def explanation_heatmap(exp, exp_class):
    '''
    Using heat-map to highlight the importance of each super-pixel for the model prediction
    '''
    dict_heatmap = dict(exp.local_exp[exp_class])
    heatmap = np.vectorize(dict_heatmap.get)(exp.segments)
    plt.imshow(heatmap, cmap = 'RdBu', vmin  = -heatmap.max(), vmax = heatmap.max())
    plt.colorbar()
    plt.show()

explanation_heatmap(exp, exp.top_labels[0])

We can clearly identify the most influential segments used by the model to make the prediction using this heatmap visualization. Now, let try to perform the same steps for another explanation class and see if the results are different.

In [ ]:
index = 2
print(f"We will deal with predicted class: {top5_pred[index][1]}")

We will deal with predicted class: hammerhead

In [ ]:
generate_prediction_sample(exp, exp.top_labels[index], weight = 0.0001, show_positive = False, hide_background = False)

In [ ]:
explanation_heatmap(exp, exp.top_labels[index])

In this case, we are trying to find out what made the model predict the outcome as hammerhead shark. When we used the LIME explaianbility methods, the visualizations clearly show that the middle part of the shark along with its fin, does contribute positively towards predicting the outcome as hammerhead shark, but the face and the front part contribute negatively towards the prediction. This is quite consistent with our human knowledge as well. Hammerhead Sharks are also sharks, so the middle part and the fin looks similar to Tiger Sharks but the face or the front portion of the hammerhead sharks looks like the shape of a hammer, which is significantly different from that of a tiger shark.

# **Task**
Try with pretrained Tensorflow models (ResNet50, VGG16, VGG19, InceptionV3 and others), observe and analyse different outcomes.Refer: https://keras.io/api/applications/ to see differnet pretrained Tensorflow model

#### Using ResNet50

In [ ]:
import keras
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

img_path = image_path
img = keras.utils.load_img(img_path, target_size=(224, 224))
img_array = keras.utils.img_to_array(img)
x = np.expand_dims(img_array, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [ ]:
# Predict class of the image
def get_model_predictions(model, data):
    model_prediction = model.predict(data)
    print(f"The predicted class is : {decode_predictions(model_prediction, top=1)[0][0][1]}")
    return decode_predictions(model_prediction, top=1)[0][0][1] # get class name

plt.imshow(img)
pred_orig = get_model_predictions(model, x)

In [ ]:
model_prediction = model.predict(x)
top5_pred = decode_predictions(model_prediction, top=5)[0]
for pred in top5_pred:
    print(pred[1])

In [ ]:
explainer = lime_image.LimeImageExplainer()

In [ ]:
exp = explainer.explain_instance(img_array.astype('double'),
                                 model.predict,
                                 top_labels=5,
                                 hide_color=0, # color code to mask
                                 num_samples=1000) # number of pertutated images

In [ ]:
plt.imshow(exp.segments)
plt.axis('off')
plt.show()

In [ ]:
generate_prediction_sample(exp, exp.top_labels[0], weight=0.04, show_positive = True, hide_background = True)

In [ ]:
explanation_heatmap(exp, exp.top_labels[0])

In [ ]:
index = 2
print(f"We will deal with predicted class: {top5_pred[index][1]}")

In [ ]:
generate_prediction_sample(exp, exp.top_labels[index], weight = 0.02, show_positive = False, hide_background = False)

In [ ]:
explanation_heatmap(exp, exp.top_labels[index])

#### Using VGG16

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

model = VGG16(weights='imagenet', include_top=False)

img = keras.utils.load_img(img_path, target_size=(224, 224))
img_array = keras.utils.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)